# Walmart sales data analysis

## Aim

To predict aggregate monthly sales using Regression models over Walmart dataset.

## Objectives
- Getting familiar with data analysis techniques
- Learning about prediction models/algorithms through experimentation

## Methodology

- Step 0: Environment Setup
- Step 1: Reading data
- Step 2: Merging and correlation analysis
- Step 3: Testing different prediction algorithms
- Step 4: Inserting small UI elements to customise algorithms and dataframes

In [ ]:
import matplotlib.pyplot as plt
import datetime
import numpy as np
import pandas as pd  
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from ipywidgets import widgets
from IPython.display import display
from calendar import month_name
import sys

import os
current_dir = os.getcwd()
sys.path.insert(1, current_dir + "/project")

import algorithm.implementation_knn as knn_i
import algorithm.implementation_rf as rf
import algorithm.implementation_dtree as lr
import utils.implementation_scaler as sc
import utils.implementation_error as er
%matplotlib inline

## Loading Data into dataframes

In [ ]:
train = pd.read_csv("./data/train.csv")
#test = pd.read_csv("./data/test.csv")
stores = pd.read_csv("./data/stores.csv")
features = pd.read_csv("./data/features.csv")

## Exploring data

In [ ]:
# finding length
print(len(features))

# finding dimentionality
features.shape

**Total rows are 8190.**

**There are twelve columns.**

In [ ]:
features.head()

In [ ]:
features.tail()

In [ ]:
stores.head()

In [ ]:
stores.tail()

In [ ]:
train.head()

In [ ]:
train.tail()

In [ ]:
features.info()

- Date is recognised as an "Object" by pandas.
- It means that it is not recognised as any pre-defined Python type

### Getting an overview of data

In [ ]:
features.describe()
# Analysis and calculations regarding quantitative columns

In [ ]:
# Including object
# Date column
features.describe(include=object)

In [ ]:
# Including object
# Date column
features.describe(include=bool)

In [ ]:
features.count()

In [ ]:
# Counting Null values
features.isna().sum()

In [ ]:
print(len(stores))
stores.isna().sum()

In [ ]:
print(len(train))
train.isna().sum()

## Cleaning the data

In [ ]:
for i in range(1, 6):
    features[f"MarkDown{i}"] = features[f"MarkDown{i}"].fillna(0)
    #features[f"MarkDown{i}"] = features[f"MarkDown{i}"].fillna(np.mean(features[f"MarkDown{i}"]))
# CPI and Unemployment will be substituted by mean
features["Unemployment"] = features["Unemployment"].fillna(np.mean(features["Unemployment"]))
features["CPI"] = features["CPI"].fillna(np.mean(features["CPI"]))

#knn_impute(target=features[''], attributes=df.drop(['Age', 'PassengerId'], 1),
#                                    aggregation_method="median", k_neighbors=10, numeric_distance='euclidean',
#                                    categorical_distance='hamming', missing_neighbors_threshold=0.8)

In [ ]:
features.head()

In [ ]:
print(len(pd.unique(features["Store"])))

In [ ]:
print(len(pd.unique(train["IsHoliday"])))

In [ ]:
main = pd.merge(train, stores, on = "Store", how = "right", sort = False, )
main = pd.merge(main, features, on = ["Store", "Date"], how = "left", sort = False)

# Sorting the dataframe by date

main = main.sort_values(by=["Date"])


main["Month"] = pd.to_datetime(main["Date"]).dt.month
main["Year"] = pd.to_datetime(main["Date"]).dt.year

main = main.sort_values(by=["Month", "Year"])


In [ ]:
main.head()

In [ ]:
# filling in missing values

#from knnimpute import optimistic
# Calculating missing mask for main["MarkDown1"]
#mm = []

#mm = main["MarkDown1"].apply(lambda e:True if e == None else False)
#type(main["MarkDown1"])
#main["MarkDown1"] = optimistic.knn_impute_optimistic(X = pd.core.series.Series(main["MarkDown1"]),missing_mask = mm ,k = 10)

In [ ]:
bar_plot = sns.barplot(x="Month", y="Weekly_Sales",  data=main)
bar_plot.set_xticklabels(bar_plot.get_xticklabels(), fontsize=8)

In [ ]:
len(main)

In [ ]:
# IsHoliday_x and IsHoliday_y are duplicates
main["IsHoliday"] = main["IsHoliday_y"]
main.drop(labels=["IsHoliday_y", "IsHoliday_x"], axis = 'columns', inplace=True)

# dropping MarkDown4
main = main.drop(labels=["MarkDown4"], axis = 'columns')

# Combining other MarkDowns into one
#main["MarkDown"] = main["MarkDown1"] + main["MarkDown2"] + main["MarkDown3"] + main["MarkDown5"]
#main = main.drop(labels=["MarkDown1", "MarkDown2", "MarkDown3", "MarkDown5"], axis = 'columns')

In [ ]:
correlation_matrix = main.corr().round(3)

figure, axes = plt.subplots(figsize=(12,12))
sns.heatmap(correlation_matrix, annot=True, linewidths=.7, ax=axes)

#### Observations

- **High positive correlation with Weekly_Sales**
  - Dept - .15
  - Size - .24
- **High negative correlation with Weekly_Sales**
  - NA

- MarkDown1 and MarkDown4 are highly correlated so only taking one of them should be sufficient. ~Also, summing up MarkDowns is a good idea.~

## Data analysis

In [ ]:
# Make a different dataframe for every month - or aggregate it
# Test rmse and r^2 for different models

In [ ]:
main.tail()

In [ ]:
# RF
# KNN
# DecisionTree

In [ ]:
X = pd.DataFrame(np.c_[main['Dept'], main['Size']], columns = ['Dept','Size'])
Y = main['Weekly_Sales']

In [ ]:
X_train, X_test, Y_train, Y_test = sc.split_data(X, Y)
print(type(Y))

In [ ]:
model = LinearRegression()
model.fit(X_train, Y_train)

In [ ]:
er.error_calculator(model, X_train, Y_train, X_test, Y_test)

In [ ]:

months = ['All','January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

"""
month_dict = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12
}
"""

toggle_buttons = widgets.ToggleButtons(
    options=['Monthly Prediction', 'Annual Prediction'],
    value='Monthly Prediction',
    description='Options',
    disabled=False,
    button_style='info',
    tooltips=['Predict monthwise sales', 'Predict sales on Annual basis']
)

toggle_buttons

In [ ]:

month_select = widgets.Dropdown(options = months, value = 'January', description = 'Month', disabled = False)
if toggle_buttons.value == 'Annual Prediction':
    month_select.disabled = True
    month_select.value = 'All'
month_select

In [ ]:
# Make dataframe according to the month that was selected
# if it was all, leave the df as it was

df = main
df.head()

if not month_select.value == 'All':
    df.set_index(keys = ['Month'], drop = False, inplace = True)
    df = df.loc[df.Month == months.index(month_select.value) + 1]
df.head()

In [ ]:
models = ["KNN", "RandomForest", "DecisionTree"]

model_select = widgets.Dropdown(options = models, value = 'KNN', description = 'Algorithms', disabled = False)

model_select

In [ ]:
df.head()

In [ ]:
X = pd.DataFrame(np.c_[#df['Store'],
                       df['Dept'],
                       df['Size'], 
                       #df['Temperature'],
                       #df['Fuel_Price'],
                       #df['CPI'],
                       #df['Unemployment'],
                       #df['Year'],
                       df['IsHoliday'],
                       df['MarkDown3'],
                       df['Type'],
                       df['Month']],
                 
                 columns = [#'Store',
                            'Dept',
                            'Size',
                            #'Temperature',
                            #'Fuel_Price',
                            #'CPI',
                            #'Unemployment',
                            #'Year',
                            'IsHoliday',
                            'MarkDown3',
                            'Type',
                            'Month'])
Y = df['Weekly_Sales']
t = pd.get_dummies(X.Type)
X = pd.concat([X, t], axis='columns')
X = X.drop(columns=['Type'])
X.info()

In [ ]:
X_train, X_test, Y_train, Y_test = sc.split_data(X, Y)
print(type(Y))

In [ ]:
X_train, X_test = sc.standard(X_train, X_test)


In [ ]:
if(model_select.value == "KNN"):
    model, y_pred = knn_i.prediction(X_train,Y_train, X_test, Y_test)
elif (model_select.value == "RandomForest"):
    model, y_pred = rf.prediction(X_train,Y_train, X_test, Y_test)
elif (model_select.value == "DecisionTree"):
    model, y_pred = lr.prediction(X_train,Y_train, X_test, Y_test)

er.error_calculator(model, X_train, Y_train, X_test, Y_test)
plt.scatter(Y_test, y_pred)

In [ ]:
X.head()

In [ ]:
pd.DataFrame(y_pred)

## References

- https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
- https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
- https://medium.com/analytics-vidhya/walmart-sales-forecasting-d6bd537e4904
- https://stackoverflow.com/questions/5137497/find-current-directory-and-files-directory
- https://stackoverflow.com/questions/19790790/splitting-dataframe-into-multiple-dataframes
- https://stackoverflow.com/questions/38913965/make-the-size-of-a-heatmap-bigger-with-seaborn
- https://datascience.stackexchange.com/questions/9159/when-to-choose-linear-regression-or-decision-tree-or-random-forest-regression